In [71]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import etree
from urllib.request import urlopen
import numpy as np

In [72]:
#Funktion, um Anbieter für Gemeinde zu bekommen
def get_provider(bfs):
    url = 'https://www.strompreis.elcom.admin.ch/municipality/'+str(bfs)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    dom = etree.HTML(str(soup))
    anbieter1 = dom.xpath('/html/body/div[1]/div/div[1]/div[1]/div[2]/div/div/a')[0].text
    try:
        anbieter2 = dom.xpath('/html/body/div[1]/div/div[1]/div[1]/div[2]/div/div/a')[1].text
        return anbieter1,anbieter2
    except:
        return anbieter1,0

#Mit diesen Funktionen werden für die Anbieter die unterschiedlichen Wert (Totalkosten + Aufteilung) herausgelesen
def get_total(anbieter):
    try:
        total = tarife[tarife['operatorLabel'] == anbieter]['total'].values[0]
    except:
        total = np.NaN
    return total

def get_netznutzung(anbieter):
    try:
        netznutzung = tarife[tarife['operatorLabel'] == anbieter]['gridusage'].values[0]
    except:
        netznutzung = np.NaN
    return netznutzung

def get_energie(anbieter):
    try:
        energie = tarife[tarife['operatorLabel'] == anbieter]['energy'].values[0]
    except:
        energie = np.NaN
    return energie

def get_abgaben_gemeinwesen(anbieter):
    try:
        abgaben_gemeinwesen = tarife[tarife['operatorLabel'] == anbieter]['charge'].values[0]
    except:
        abgaben_gemeinwesen = np.NaN
    return abgaben_gemeinwesen

def get_foerderabgabgen(anbieter):
    try:
        foerderabgabgen = tarife[tarife['operatorLabel'] == anbieter]['aidfee'].values[0]
    except:
        foerderabgabgen = np.NaN
    return foerderabgabgen

#Wenn eine Gemeinde zwei Anbieter hat (mehr als zwei pro Gemeinde gibt's im Aargau bislang nicht) dann wird der
#Durchschnittspreis berechnet. Wenn nur 1 Anbieter den entsprechenden Tarif hat, dann wird dieser zurückgegeben.
def get_durchschnitt(anbieter1,anbieter2):
    if np.isnan(anbieter1):
        return anbieter2
    elif np.isnan(anbieter2):
        return anbieter1
    else:
        durchschnitt = (anbieter1+anbieter2)/2
        return durchschnitt

    

In [82]:
#Hier wird die Kategorie bestimmt. Letzts Mal: H1,H4 und H8. 
#Kategorie bestimmen und Programm laufen lassen, dann wird ein CSV mit dem Kategorienamen ausgegeben.
category = "H8"

#Das CSV mit den aktuellen Tarifen. Kann hier herungergeladen werden: https://www.strompreis.elcom.admin.ch/
tarife = pd.read_csv("tarife-2022.csv")
tarife = tarife[(tarife['product'] == "standard") & (tarife['category'] == category)]

#Das CSV mit den Gemeinden und Wappen
gemeinden = pd.read_csv("bfs-wappen.csv")

In [83]:
#Neue Liste mit Minidict machen, für jede Gmeinde BFS, Wappen und Anbieter reinschreiben
AG_Anbieter = []

for bfs in gemeinden['bfs']:
    name = gemeinden[gemeinden['bfs'] == bfs]['Gemeinde'].values[0]
    wappen = gemeinden[gemeinden['bfs'] == bfs]['wappen'].values[0]
    anbieter1,anbieter2=get_provider(bfs)
    mini_dict = {"BFS":bfs, "Gemeinde":name, "Wappen":wappen, "Anbieter 1":anbieter1, "Anbieter 2":anbieter2}
    AG_Anbieter.append(mini_dict)
    
list_AG_Anbieter = pd.DataFrame(AG_Anbieter)

In [84]:
#Mit Hilfe der Funktionen die Preise für die jeweiligen Anbieter reinschreiben
list_AG_Anbieter['total'] = list_AG_Anbieter['Anbieter 1'].apply(get_total)
list_AG_Anbieter['netznutzung'] = list_AG_Anbieter['Anbieter 1'].apply(get_netznutzung)
list_AG_Anbieter['energie'] = list_AG_Anbieter['Anbieter 1'].apply(get_energie)
list_AG_Anbieter['abgaben_gemeinwesen'] = list_AG_Anbieter['Anbieter 1'].apply(get_abgaben_gemeinwesen)
list_AG_Anbieter['foerderabgabgen'] = list_AG_Anbieter['Anbieter 1'].apply(get_foerderabgabgen)

In [85]:
list_AG_Anbieter['total2'] = list_AG_Anbieter['Anbieter 2'].apply(get_total)
list_AG_Anbieter['netznutzung2'] = list_AG_Anbieter['Anbieter 2'].apply(get_netznutzung)
list_AG_Anbieter['energie2'] = list_AG_Anbieter['Anbieter 2'].apply(get_energie)
list_AG_Anbieter['abgaben_gemeinwesen2'] = list_AG_Anbieter['Anbieter 2'].apply(get_abgaben_gemeinwesen)
list_AG_Anbieter['foerderabgabgen2'] = list_AG_Anbieter['Anbieter 2'].apply(get_foerderabgabgen)

In [86]:
#Bereinigen: 0 ersetzen, Teest-Netzbetreiber (Aarau) rauslöschen
list_AG_Anbieter=list_AG_Anbieter.replace(0, np.NaN)
list_AG_Anbieter=list_AG_Anbieter.replace("Test-Netzbetreiber AG", np.NaN)

#Angezeigter Wert berechnen
list_AG_Anbieter['angezeigter_wert'] = list_AG_Anbieter.apply(lambda x: get_durchschnitt(x['total'], x['total2']), axis=1)

In [87]:
#Resultat in CSV abspeichern für Datawrapper
list_AG_Anbieter.to_csv(category+'.csv', index=False)

# Für Datawrapper

https://app.datawrapper.de/map/7rWX8/publish

Das ist der HTML-Code für die Tooletipps. Je nach Zahlengrösse muss der Vergrösserungsfaktor angepasst werden, also z.B. *3.5 mit *4.5 ersetzen (in der letzten Grafik habe ich 3.5 für H1 und 4.5 für H4 benutzt.

{{ total!= 0 ? '<h3>':''}}{{ total!= 0 ? anbieter_1:''}}{{ total!= 0 ? '</h3>':''}}{{ total!= 0 ? '<table><tr><td><b>Kosten total</b><b></td><td><b>':''}}{{ total!= 0 ? total:''}}{{ total!= 0 ? ' Rp./kWh</b></td></tr>':''}}{{ total!= 0 ? '<td colspan="2">Aufteilung (Angaben in Rp./kWh):</td><tr><td>Netznutzung</td><td><div style="width:':''}}{{ total!= 0 ? netznutzung*3.5 :''}}{{ total!= 0 ? 'px; height:15px; background-color:#1d81a2; color:white; padding:4px 4px 0px 4px; vertical-align:bottom; font-weight:bold; display:inline-block;"></div><div style="width:':''}}{{ total!= 0 ? 80-(netznutzung*3.5):''}}{{ total!= 0 ? 'px; height:15px; background-color:#F2F2F2; color:#1d81a2; vertical-align:bottom; padding:4px 4px 0px 4px; font-weight:bold; display:inline-block;">':''}}{{ total!= 0 ? FORMAT(netznutzung, "0.00"):''}}{{ total!= 0 ? '</div></td></tr>':''}}{{ total!= 0 ? '<tr><td>Energie</td><td><div style="width:':''}}{{ total!= 0 ? energie*3.5:''}}{{ total!= 0 ? 'px; height:15px; background-color:#1d81a2; color:white; padding:4px 4px 0px 4px; vertical-align:bottom; font-weight:bold; display:inline-block;"></div><div style="width:':''}}{{ total!= 0 ? 80-(energie*3.5):''}}{{ total!= 0 ? 'px; height:15px; background-color:#F2F2F2; color:#1d81a2; vertical-align:bottom; padding:4px 4px 0px 4px; font-weight:bold; display:inline-block;">':''}}{{ total!= 0 ? FORMAT(energie, "0.00"):''}}{{ total!= 0 ? '</div></td></tr>':''}}{{ total!= 0 ? '<tr><td>Gemeinwesen</td><td><div style="width:':''}}{{ total!= 0 ? abgaben_gemeinwesen*3.5:''}}{{ total!= 0 ? 'px; height:15px; background-color:#1d81a2; color:white; padding:4px 4px 0px 4px; vertical-align:bottom; font-weight:bold; display:inline-block;"></div><div style="width:':''}}{{ total!= 0 ? 80-(abgaben_gemeinwesen*3.5):''}}{{ total!= 0 ? 'px; height:15px; background-color:#F2F2F2; color:#1d81a2; vertical-align:bottom; padding:4px 4px 0px 4px; font-weight:bold; display:inline-block;">':''}}{{ total!= 0 ? FORMAT(abgaben_gemeinwesen, "0.00"):''}}{{ total!= 0 ? '</div></td></tr>':''}}{{ total!= 0 ? '<tr><td>Förderabg.</td><td><div style="width:':''}}{{ total!= 0 ? foerderabgabgen*3.5:''}}{{ total!= 0 ? 'px; height:15px; background-color:#1d81a2; color:white; padding:4px 4px 0px 4px; vertical-align:bottom; font-weight:bold; display:inline-block;"></div><div style="width:':''}}{{ total!= 0 ? 80-(foerderabgabgen*3.5):''}}{{ total!= 0 ? 'px; height:15px; background-color:#F2F2F2; color:#1d81a2; vertical-align:bottom; padding:4px 4px 0px 4px; font-weight:bold; display:inline-block;">':''}}{{ total!= 0 ? FORMAT(foerderabgabgen, "0.00"):''}}{{ total!= 0 ? '</div></td></tr></table>':''}}
{{ total2!= 0 ? '<h3>':''}}{{ total2!= 0 ? anbieter_2:''}}{{ total2!= 0 ? '</h3>':''}}{{ total2!= 0 ? '<table><tr><td><b>Kosten total</b><b></td><td><b>':''}}{{ total2!= 0 ? total2:''}}{{ total2!= 0 ? ' Rp./kWh</b></td></tr>':''}}{{ total2!= 0 ? '<td colspan="2">Aufteilung (Angaben in Rp./kWh):</td><tr><td>Netznutzung</td><td><div style="width:':''}}{{ total2!= 0 ? netznutzung2*3.5 :''}}{{ total2!= 0 ? 'px; height:15px; background-color:#1d81a2; color:white; padding:4px 4px 0px 4px; vertical-align:bottom; font-weight:bold; display:inline-block;"></div><div style="width:':''}}{{ total2!= 0 ? 80-(netznutzung2*3.5):''}}{{ total2!= 0 ? 'px; height:15px; background-color:#F2F2F2; color:#1d81a2; vertical-align:bottom; padding:4px 4px 0px 4px; font-weight:bold; display:inline-block;">':''}}{{ total2!= 0 ? FORMAT(netznutzung2, "0.00"):''}}{{ total2!= 0 ? '</div></td></tr>':''}}{{ total2!= 0 ? '<tr><td>Energie</td><td><div style="width:':''}}{{ total2!= 0 ? energie2*3.5:''}}{{ total2!= 0 ? 'px; height:15px; background-color:#1d81a2; color:white; padding:4px 4px 0px 4px; vertical-align:bottom; font-weight:bold; display:inline-block;"></div><div style="width:':''}}{{ total2!= 0 ? 80-(energie2*3.5):''}}{{ total2!= 0 ? 'px; height:15px; background-color:#F2F2F2; color:#1d81a2; vertical-align:bottom; padding:4px 4px 0px 4px; font-weight:bold; display:inline-block;">':''}}{{ total2!= 0 ? FORMAT(energie2, "0.00"):''}}{{ total2!= 0 ? '</div></td></tr>':''}}{{ total2!= 0 ? '<tr><td>Gemeinwesen</td><td><div style="width:':''}}{{ total2!= 0 ? abgaben_gemeinwesen2*3.5:''}}{{ total2!= 0 ? 'px; height:15px; background-color:#1d81a2; color:white; padding:4px 4px 0px 4px; vertical-align:bottom; font-weight:bold; display:inline-block;"></div><div style="width:':''}}{{ total2!= 0 ? 80-(abgaben_gemeinwesen2*3.5):''}}{{ total2!= 0 ? 'px; height:15px; background-color:#F2F2F2; color:#1d81a2; vertical-align:bottom; padding:4px 4px 0px 4px; font-weight:bold; display:inline-block;">':''}}{{ total2!= 0 ? FORMAT(abgaben_gemeinwesen, "0.00"):''}}{{ total2!= 0 ? '<tr><td>Förderabg.</td><td><div style="width:':''}}{{ total2!= 0 ? foerderabgabgen2*3.5:''}}{{ total2!= 0 ? 'px; height:15px; background-color:#1d81a2; color:white; padding:4px 4px 0px 4px; vertical-align:bottom; font-weight:bold; display:inline-block;"></div><div style="width:':''}}{{ total2!= 0 ? 80-(foerderabgabgen2*3.5):''}}{{ total2!= 0 ? 'px; height:15px; background-color:#F2F2F2; color:#1d81a2; vertical-align:bottom; padding:4px 4px 0px 4px; font-weight:bold; display:inline-block;">':''}}{{ total2!= 0 ? FORMAT(foerderabgabgen2, "0.00"):''}}{{ total2!= 0 ? '</div></td></tr></table>':''}}
    
    
##Tabelle mit Tarifen
    
Bei Description für jeden Tarif die entsprechende Tabelle anpassen, falls sie nicht gleich geblieben ist:
    
Wählen Sie einen Tarif: H1 /<b> H4 </b>/ H8
<br><br>
<i>Viele Stromlieferanten haben keinen Einheitspreis für alle Kunden, sondern machen ihre Tarife von der Menge und dem Zeitpunkt des Stromkonsums abhängig. Die Tarifstrukturen können komplex und von Lieferant zu Lieferant unterschiedlich sein. Um trotzdem einen Preisvergleich zu ermöglichen, rechnen wir mit Verbraucherprofilen, die wir vom Strompreisrechner des Bundes übernommen haben.</i>
<br><br>
<table width="100%"><tr><td colspan="2"><b>H4: 5-Zimmerwohnung mit Elektroherd und Tumbler (ohne Elektroboiler. Gesamtverbrauch pro Jahr von 4'500 kWh, aufgeteilt wie folgt:</b></td></tr>
<tr bgcolor=#9ADEDE><td rowspan="4">Winter</td><td>06h-12h: 690 kWh</td></tr>
<tr bgcolor=#9ADEDE><td>12h-18h: 630 kWh</td></tr>
<tr bgcolor=#9ADEDE><td>18h-22h	600 kWh</td></tr>
<tr bgcolor=#9ADEDE><td>22h-06h	600 kWh</td></tr>
<tr bgcolor=#FEF593><td rowspan="4">Sommer</td>
<td>06h-12h	600 kWh</td></tr>
<tr bgcolor=#FEF593><td>12h-18h	540 kWh</td></tr>
<tr bgcolor=#FEF593><td>18h-22h	500 kWh</td></tr>
<tr bgcolor=#FEF593><td>22h-06h	440 kWh</td></tr></table><br>